# 🚀 Улучшенный BERT+CRF для NER с достижением macro-F1 ≥ 0.8

Полная реализация с word-level токенизацией, CRF слоем и постпроцессингом для VOLUME/PERCENT

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/your-repo/bert-crf-ner.ipynb)

---

## 📦 1. Установка зависимостей

In [1]:
!pip install pyspellchecker pymorphy3 rapidfuzz evaluate seqeval fuzzywuzzy pytorch-crf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 96.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 144.5 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=91957c907f250e915be1f830db1cdd365422376f97a2936f7227ccc094356151
  Stored in directory: /root/.cache/pip/wheels/5f/b8/73/0b2c1a76b701a677653dd79ece07cfabd7457989dbfbdcd8d7
Successfully built seqeval


In [2]:
!pip install --upgrade accelerate

In [3]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 131.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.1
    Uninstalling transformers-4.56.1:
      Successfully uninstalled transformers-4.56.1


In [4]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/lct_2025/')

Mounted at /content/drive


In [5]:
import numpy as np
import pandas as pd
import random
import re
import json
import asyncio
import pymorphy3
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments, pipeline
from transformers import BertConfig, BertForTokenClassification, BertTokenizerFast, pipeline, BertModel
from transformers import EarlyStoppingCallback
from datasets import Dataset
from sklearn.metrics.pairwise import cosine_similarity
import torch
from sklearn.preprocessing import normalize
from fuzzywuzzy import process
from sklearn.cluster import KMeans
from sklearn.metrics import f1_score, precision_score, recall_score
from ast import literal_eval
import evaluate
from functools import partial

/usr/local/lib/python3.12/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [6]:
# Импорты
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.optim import AdamW
import re
import ast
import random
import json
import os
from collections import Counter, defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
import string
from rapidfuzz import process, fuzz
warnings.filterwarnings('ignore')

# Transformers
from transformers import (
    AutoTokenizer, AutoModel, AutoConfig,
    Trainer, TrainingArguments,
    DataCollatorForTokenClassification,
    get_linear_schedule_with_warmup
)

# Datasets и метрики
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report
import evaluate
from torchcrf import CRF

# Настройка для воспроизводимости
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

# Проверка GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🔥 PyTorch версия: {torch.__version__}")
print(f"💻 Устройство: {device}")
if torch.cuda.is_available():
    print(f"🚀 GPU: {torch.cuda.get_device_name(0)}")
    print(f"📊 CUDA память: {torch.cuda.get_device_properties(0).total_memory // 1024**3} GB")

print("\n✅ Все импорты выполнены успешно!")

🔥 PyTorch версия: 2.8.0+cu126
💻 Устройство: cuda
🚀 GPU: Tesla T4
📊 CUDA память: 14 GB

✅ Все импорты выполнены успешно!


## ⚙️ 2. Конфигурация и паттерны

In [7]:
class Config:
    # Модель и параметры
    MODEL_NAME = "./tiny2_v5"
    MAX_LENGTH = 128
    BATCH_SIZE = 8  # Уменьшили для стабильности
    OUTPUT_DIR = "./bert-bilstm-crf-fixed"

    # Обучение
    EPOCHS = 3
    LEARNING_RATE = 1e-5  # Уменьшили
    BILSTM_LEARNING_RATE = 2e-5  # Немного выше для BiLSTM
    CRF_LEARNING_RATE = 1e-5
    WEIGHT_DECAY = 0.01
    WARMUP_RATIO = 0.1
    GRADIENT_ACCUMULATION_STEPS = 4

    # BiLSTM параметры
    BILSTM_HIDDEN_SIZE = 256  # Уменьшили с 512
    BILSTM_NUM_LAYERS = 1     # Уменьшили с 2
    BILSTM_DROPOUT = 0.3

    # Метки - ВАЖНО: правильный порядок
    LABELS = ["O", "B-TYPE", "I-TYPE", "B-BRAND", "I-BRAND"]
    LABEL_TO_ID = {label: idx for idx, label in enumerate(LABELS)}
    ID_TO_LABEL = {idx: label for label, idx in LABEL_TO_ID.items()}

    # 🔍 Расширенные VOLUME паттерны с полными склонениями и ошибками
    VOLUME_PATTERNS = [
        # ЛИТРЫ - все падежи и варианты написания
        r'\b(\d+(?:[.,]\d+)?)\s*(?:литр|литра|литров|литре|литрам|литрами|литрах|литру|литром|л|l)\b',
        r'\b(\d+(?:[.,]\d+)?)\s*(?:литер|литера|литеров|литеры|литеру|литером|литере|литерам|литерами)\b',
        r'\b(\d+(?:[.,]\d+)?)\s*(?:лит|лита|литы|литов|литу|литом)\b',

        # ГРАММЫ - все падежи и варианты с ошибками
        r'\b(\d+(?:[.,]\d+)?)\s*(?:г|гр|грамм|грамма|граммов|граммы|грамму|граммом|грамме|граммам|граммами|граммах)\b',
        r'\b(\d+(?:[.,]\d+)?)\s*(?:грам|грама|грамов|грамы|граму|грамом|граме|грамам|грамами|грамах)\b',
        r'\b(\d+(?:[.,]\d+)?)\s*(?:грм|грма|грмов|грмы|грму|грмом|грме)\b',

        # КИЛОГРАММЫ - все падежи и варианты с ошибками
        r'\b(\d+(?:[.,]\d+)?)\s*(?:кг|килограмм|килограмма|килограммов|килограммы|килограмму|килограммом|килограмме|килограммам|килограммами|килограммах)\b',
        r'\b(\d+(?:[.,]\d+)?)\s*(?:килаграм|килаграмма|килаграммов|килаграммы|килаграмму|килаграммом|килаграмме)\b',
        r'\b(\d+(?:[.,]\d+)?)\s*(?:киллограм|киллограмма|киллограммов|киллограммы)\b',
        r'\b(\d+(?:[.,]\d+)?)\s*(?:кило|килло)\b',

        # МИЛЛИЛИТРЫ - все падежи и ошибки
        r'\b(\d+(?:[.,]\d+)?)\s*(?:мл|ml|миллилитр|миллилитра|миллилитров|миллилитры|миллилитру|миллилитром|миллилитре)\b',
        r'\b(\d+(?:[.,]\d+)?)\s*(?:милитр|милитра|милитров|милитры|милитру|милитром|милитре)\b',
        r'\b(\d+(?:[.,]\d+)?)\s*(?:милилитр|милилитра|милилитров|милилитры)\b',
        r'\b(\d+(?:[.,]\d+)?)\s*(?:милл|мила)\b',

        # ШТУКИ и УПАКОВКИ с ошибками
        r'\b(\d+(?:[.,]\d+)?)\s*(?:шт|штук|штука|штуки|штуку|штукой|штуке|штукам|штуками|штуках)\b',
        r'\b(\d+(?:[.,]\d+)?)\s*(?:штукы|штукы|штуку|штукой)\b',
        r'\b(\d+(?:[.,]\d+)?)\s*(?:упак|упаковк|упаковка|упаковки|упаковок|упаковку|упаковкой|упаковке)\b',
        r'\b(\d+(?:[.,]\d+)?)\s*(?:упоковка|упоковки|упоковок)\b',

        # ДОПОЛНИТЕЛЬНЫЕ форматы
        r'\b(\d+(?:[.,]\d+)?)\s*[x×х]\s*(\d+(?:[.,]\d+)?)\b',
        r'\b(\d+/\d+|\d+,\d+|\d+\.\d+)\s*(?:л|литр|мл|г|грамм|кг|килограмм)\b'
    ]

    # 📈 Расширенные PERCENT паттерны с учетом ошибок
    PERCENT_PATTERNS = [
        r'\b(\d+(?:[.,]\d+)?)\s*%',
        r'\b(\d+(?:[.,]\d+)?)\s*(?:процент|процента|процентов|процентный|процентная|процентное)\b',
        r'\b(\d+(?:[.,]\d+)?)\s*(?:проц|процентов|прцент|прцентов|прцент)\b',
        r'\b(\d+(?:[.,]\d+)?)\s*(?:жирн|жирност|жирности|жирностью|жирностей)\b',
        r'\b(\d+(?:[.,]\d+)?)\s*(?:жырн|жырност|жырности)\b',
        r'\b(\d+(?:[.,]\d+)?)\s*(?:град|градус|градуса|градусов|градусы|°|гр\.)\b',
        r'\b(\d+(?:[.,]\d+)?)\s*(?:грдус|грдуса|грдусов|грдусы)\b',
        r'\b(\d+(?:[.,]\d+)?)\s*(?:алк|алкоголь|алкогольный|алкогольная|алкогольное)\b',
        r'\b(\d+(?:[.,]\d+)?)\s*(?:алког|алкогол|алкогольны)\b',
        r'\b(\d+(?:[.,]\d+)?)\s*(?:концентрац|концентрация|концентрации|концентрацией|концентраций)\b',
        r'\b(\d+(?:[.,]\d+)?)\s*(?:канцентрац|канцентрация|канцентрации)\b',
        r'\b(\d+(?:[.,]\d+)?)\s*(?:содержан|содержание|содержания|содержанием|содержаний)\b'
    ]

    # Standalone числа
    STANDALONE_NUMBER_PATTERN = r'\b(\d+(?:[.,]\d+)?)\b'

    # Исключения для standalone чисел
    EXCLUDE_PATTERNS = [
        r'\b(19\d{2}|20\d{2})\b',  # года
        r'\b\d{4}\s*(?:год|года|годов|году|годом|годе)\b',
        r'\b(?:номер|код|артикул|арт|№)\s*\d+\b',  # номера
        r'\b\d+\s*(?:руб|рубл|рублей|рубля|р\.|₽|долл|доллар|\$|€|евро)\b',  # цены
        r'\b\d{1,2}:\d{2}\b',  # время
        r'\b\d{1,2}[./]\d{1,2}[./]\d{2,4}\b'  # даты
    ]

config = Config()

def is_standalone_volume(text, number_match):
    """Проверяет, является ли отдельно стоящее число VOLUME сущностью."""
    number_start, number_end = number_match.span()
    number_text = number_match.group()

    # Проверяем исключения
    for exclude_pattern in config.EXCLUDE_PATTERNS:
        for exclude_match in re.finditer(exclude_pattern, text, re.IGNORECASE):
            if exclude_match.start() <= number_start < exclude_match.end():
                return False

    # Проверяем контекст (в радиусе 15 символов)
    context_start = max(0, number_start - 15)
    context_end = min(len(text), number_end + 15)
    context = text[context_start:context_end]

    # Проверяем VOLUME/PERCENT паттерны в контексте
    for pattern in config.VOLUME_PATTERNS + config.PERCENT_PATTERNS:
        if re.search(pattern, context, re.IGNORECASE):
            return False

    # Проверяем размер числа
    try:
        num_value = float(number_text.replace(',', '.'))
        if num_value > 10000 or num_value == 0:
            return False
    except ValueError:
        pass

    return True

print("✅ Конфигурация загружена!")
print(f"📊 Метки для обучения: {config.LABELS}")
print(f"🔍 VOLUME паттернов: {len(config.VOLUME_PATTERNS)}")
print(f"📈 PERCENT паттернов: {len(config.PERCENT_PATTERNS)}")
print(f"⚙️ Batch size: {config.BATCH_SIZE}")
print(f"🎯 Цель: macro-F1 ≥ 0.8")

✅ Конфигурация загружена!
📊 Метки для обучения: ['O', 'B-TYPE', 'I-TYPE', 'B-BRAND', 'I-BRAND']
🔍 VOLUME паттернов: 20
📈 PERCENT паттернов: 12
⚙️ Batch size: 8
🎯 Цель: macro-F1 ≥ 0.8


## 📂 3. Загрузка и обработка данных



In [8]:
'''data = pd.read_csv("./train.csv", sep=';')
new_br = pd.read_csv("./brands_pyaterochka_with_annotation.csv", sep=';')
res = pd.concat([data, new_br], ignore_index=True)
res.to_csv('./train_filled.csv', sep=';', index=False)'''

'data = pd.read_csv("./train.csv", sep=\';\')\nnew_br = pd.read_csv("./brands_pyaterochka_with_annotation.csv", sep=\';\')\nres = pd.concat([data, new_br], ignore_index=True)\nres.to_csv(\'./train_filled.csv\', sep=\';\', index=False)'

In [9]:
import pandas as pd
import ast
import random
from collections import Counter, defaultdict
from tqdm import tqdm

def clean_labels(labels):
    """
    Заменяет русскую букву 'В' на латинскую 'B' и корректирует некорректные метки 'B-O' и 'I-O'.
    """
    cleaned = []
    for lab in labels:
        lab = lab.replace("В-", "B-").replace("В", "B")
        if lab in ("B-O", "I-O"):
            lab = "O"
        cleaned.append(lab)
    return cleaned

def load_and_convert_data(file_path):
    """
    Загружает CSV с колонками 'sample' и 'annotation',
    конвертирует VOLUME/PERCENT→O и чистит метки.
    Возвращает DataFrame с колонками ['sample', 'annotation'].
    """
    try:
        df = pd.read_csv(file_path, sep=';')
    except:
        try:
            df = pd.read_csv(file_path, sep=',')
        except:
            df = pd.read_csv(file_path)

    records = []
    for _, row in tqdm(df.iterrows(), total=len(df), desc="🔄 Converting"):
        text = row['sample']
        ann = row['annotation']
        try:
            ann = ast.literal_eval(ann) if isinstance(ann, str) else ann
        except:
            continue

        converted = []
        for start, end, label in ann:
            if label.startswith(('B-VOLUME','I-VOLUME','B-PERCENT','I-PERCENT')):
                new_label = 'O'
            else:
                new_label = label
            converted.append((start, end, new_label))

        raw_labels = [lab for _, _, lab in converted]
        cleaned = clean_labels(raw_labels)
        converted_cleaned = [
            (s, e, cleaned[i]) for i, (s, e, _) in enumerate(converted)
        ]

        records.append({
            'sample': text,
            'annotation': converted_cleaned
        })

    result_df = pd.DataFrame(records)
    print(f"📊 Loaded and converted: {len(result_df):,} samples")
    return result_df

def balance_by_duplication_df(
    df,
    annotation_col='annotation',
    min_count=2000,
    max_dup=10,
    random_seed=42,
    min_brand_examples=3
):
    """
    Балансирует DataFrame за счет дублирования строк целиком
    для редких сущностей и добавляет отдельные примеры для редких брендов,
    чтобы каждый редкий бренд присутствовал минимум в min_brand_examples строках.
    """
    random.seed(random_seed)
    # частоты всех меток (кроме 'O')
    label_counts = Counter()
    for ann in df[annotation_col]:
        labels = {lab for _, _, lab in ann if lab != 'O'}
        for lab in labels:
            label_counts[lab] += 1

    balanced_indices = list(df.index)
    brand_examples = defaultdict(list)
    brand_counts = Counter()

    # собираем статистику по брендам
    for idx, row in df.iterrows():
        text = row['sample']
        ann = row[annotation_col]
        for start, end, lbl in ann:
            if 'BRAND' in lbl:
                brand = text[start:end].strip()
                brand_counts[brand] += 1
                brand_examples[brand].append(idx)

    print(f"📊 Found unique brands: {len(brand_counts)}")

    # дублирование полных строк для редких меток
    for idx, row in df.iterrows():
        ann = row[annotation_col]
        labels = {lab for _, _, lab in ann if lab != 'O'}
        if not labels:
            continue
        freqs = [label_counts[lab] for lab in labels]
        if min(freqs) < min_count:
            balanced_indices.extend([idx] * max_dup)

    balanced_df = df.loc[balanced_indices].reset_index(drop=True)

    # добавляем отдельные примеры для каждого редкого бренда
    extra = []
    for brand, count in brand_counts.items():
        if count < min_count:
            needed = max(0, min_brand_examples - count)
            for _ in range(needed):
                words = brand.split()
                spans = []
                pos = 0
                for i, w in enumerate(words):
                    start = pos
                    end = pos + len(w)
                    tag = 'B-BRAND' if i == 0 else 'I-BRAND'
                    spans.append((start, end, tag))
                    pos = end + 1
                extra.append({'sample': brand, 'annotation': spans})

    if extra:
        balanced_df = pd.concat([balanced_df, pd.DataFrame(extra)], ignore_index=True)

    balanced_df = balanced_df.sample(frac=1, random_state=random_seed).reset_index(drop=True)
    print(f"📊 Before balance: {len(df):,}, After: {len(balanced_df):,}")
    return balanced_df

from collections import defaultdict

def stratified_split_by_entities(df, test_size=0.2, random_state=42):
    """
    Разбивает df на train и validation, гарантируя, что каждый бренд и каждая сущность
    присутствуют хотя бы в одной строке в обеих выборках. Возвращает train_df, val_df.
    """
    df = df.copy().reset_index(drop=True)
    rng = np.random.RandomState(random_state)
    # 1. Первичное случайное разбиение
    idxs = np.arange(len(df))
    train_idx, val_idx = train_test_split(
        idxs, test_size=test_size, random_state=rng.randint(1e6)
    )
    train_set = set(train_idx)
    val_set = set(val_idx)

    # 2. Собираем индексы по брендам и по другим меткам
    brand_to_idxs = defaultdict(list)
    label_to_idxs = defaultdict(list)
    for i, row in df.iterrows():
        text = row['sample']
        for s, e, label in row['annotation']:
            if label != 'O':
                label_to_idxs[label].append(i)
            if 'BRAND' in label:
                brand = text[s:e]
                brand_to_idxs[brand].append(i)

    # 3. Для каждого бренда, если в val нет ни одного примера, перенести один из train
    for brand, ids in brand_to_idxs.items():
        if not val_set.intersection(ids):
            # выбрать случайный id бренда из train
            candidates = [i for i in ids if i in train_set]
            if candidates:
                chosen = rng.choice(candidates)
                train_set.remove(chosen)
                val_set.add(chosen)

    # 4. Аналогично для каждой сущностной метки (TYPE, etc.)
    for label, ids in label_to_idxs.items():
        if not val_set.intersection(ids):
            candidates = [i for i in ids if i in train_set]
            if candidates:
                chosen = rng.choice(candidates)
                train_set.remove(chosen)
                val_set.add(chosen)

    # 5. Итоговые DataFrame
    train_df = df.loc[sorted(train_set)].reset_index(drop=True)
    val_df   = df.loc[sorted(val_set)].reset_index(drop=True)
    print(f"📊 Train: {len(train_df)}, Validation: {len(val_df)}")
    return train_df, val_df

# Пайплайн:
# 1) Загрузить и конвертировать:
df_conv = load_and_convert_data("./train_big.csv")
# 2) Сбалансировать дублированием и добавить редкие бренды:
df_bal = balance_by_duplication_df(
    df_conv,
    max_dup=6,
    min_count=2000,
    min_brand_examples=3
)


🔄 Converting: 100%|██████████| 38870/38870 [00:11<00:00, 3411.23it/s]


📊 Loaded and converted: 38,870 samples
📊 Found unique brands: 4650
📊 Before balance: 38,870, After: 38,912


In [10]:
train, val = stratified_split_by_entities(df_bal, test_size=0.05)

📊 Train: 33334, Validation: 5578


In [11]:
train

,sample,annotation
0,свикла,"[(0, 6, B-TYPE)]"
1,сливови,"[(0, 7, B-TYPE)]"
2,молоко безлак,"[(0, 6, B-TYPE), (7, 13, I-TYPE)]"
3,цыпленок,"[(0, 8, B-TYPE)]"
4,ячм,"[(0, 3, B-TYPE)]"
...,...,...
33329,крючки,"[(0, 6, B-TYPE)]"
33330,greenfield,"[(0, 10, B-BRAND)]"
33331,топленное,"[(0, 9, B-TYPE)]"
33332,питаха,"[(0, 6, B-TYPE)]"


In [12]:
train.groupby(by=['sample']).count().sort_values(by='annotation')

,annotation
sample,
яьлоко,1
яюколбаса со шпиком,1
яюлоко,1
яяблоки,1
яяблоко,1
...,...
вкус & польза,14
сады придонья,14
русская картошка,14


In [13]:
val.groupby(by=['sample']).count().sort_values(by='annotation')

,annotation
sample,
наггетцы sadi,1
напиток laimon fresh,1
напиток fresh aloe,1
напито,1
напитки сокосодержащие,1
...,...
кефире бежит луг,2
булочка с абрикос,3
сады придонья мультифрукт,3


In [14]:
train[train['sample'] == 'красный октябрь']

,sample,annotation
125,красный октябрь,"[(0, 7, B-BRAND), (8, 15, I-BRAND)]"
7686,красный октябрь,"[(0, 7, B-BRAND), (8, 15, I-BRAND)]"
10901,красный октябрь,"[(0, 7, B-BRAND), (8, 15, I-BRAND)]"
14486,красный октябрь,"[(0, 7, B-BRAND), (8, 15, I-BRAND)]"
15210,красный октябрь,"[(0, 7, B-BRAND), (8, 15, I-BRAND)]"
15546,красный октябрь,"[(0, 7, B-BRAND), (8, 15, I-BRAND)]"
19387,красный октябрь,"[(0, 7, B-BRAND), (8, 15, I-BRAND)]"
19930,красный октябрь,"[(0, 7, B-BRAND), (8, 15, I-BRAND)]"
21424,красный октябрь,"[(0, 7, B-BRAND), (8, 15, I-BRAND)]"
23412,красный октябрь,"[(0, 7, B-BRAND), (8, 15, I-BRAND)]"


In [15]:
val[val['sample'] == 'красный октябрь']

,sample,annotation
1509,красный октябрь,"[(0, 7, B-BRAND), (8, 15, I-BRAND)]"


In [16]:
# =============================================
# 🔤 ОБРАБОТКА СИМВОЛОВ И ОПЕЧАТОК
# =============================================

# Создание символьного словаря
CHAR_VOCAB = ['<PAD>'] + sorted(set(string.ascii_letters + string.digits +
                                    'абвгдеёжзийклмнопрстуфхцчшщъыьэюя' +
                                    'АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ' +
                                    '.,!?;:-_()[]{}/*+=%@#$&^~`|\\\'\"'))
CHAR_TO_IDX = {char: idx for idx, char in enumerate(CHAR_VOCAB)}

def char_tensor(token, max_len=20):
    """Преобразует токен в тензор символов"""
    ids = [CHAR_TO_IDX.get(c, 0) for c in token[:max_len]]
    return ids + [0] * (max_len - len(ids))

def augment_typo(token, prob=0.15):
    """Генерирует опечатки в токене"""
    if random.random() > prob or len(token) < 2:
        return token

    ru_keyboard = {
        'й':'f', 'ц':'w', 'у':'e', 'к':'r', 'е':'t', 'н':'y', 'г':'u', 'ш':'i',
        'щ':'o', 'з':'p', 'х':'[', 'ъ':']', 'ф':'a', 'ы':'s', 'в':'d',
        'а':'f', 'п':'g', 'р':'h', 'о':'j', 'л':'k', 'д':'l', 'ж':';',
        'э':"'", 'я':'z', 'ч':'x', 'с':'c', 'м':'v', 'и':'b', 'т':'n',
        'ь':'m', 'б':',', 'ю':'.', '.':', '
    }

    ops = random.choice(['swap', 'delete', 'insert', 'qwerty'])
    s = list(token)

    if ops == 'swap' and len(s) > 2:
        i = random.randrange(len(s)-1)
        s[i], s[i+1] = s[i+1], s[i]
    elif ops == 'delete' and len(s) > 3:
        del s[random.randrange(len(s))]
    elif ops == 'insert':
        s.insert(random.randrange(len(s)), random.choice(string.ascii_lowercase))
    else:  # qwerty replace
        i = random.randrange(len(s))
        repl = ru_keyboard.get(s[i].lower())
        if repl: s[i] = repl

    return ''.join(s)

# Фаззи-матчинг для брендов
def fuzzy_brand_match(token, brand_vocab, threshold=85):
    """Исправляет бренд через fuzzy matching"""
    if len(token) < 3:
        return token

    match = process.extractOne(
        token, brand_vocab, scorer=fuzz.ratio, score_cutoff=threshold
    )
    return match[0] if match else token

print("✅ Функции обработки символов готовы!")


✅ Функции обработки символов готовы!


In [17]:
class BiLSTMDataCollatorForTokenClassification:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def __call__(self, features):
        batch = {}

        # Стандартные поля
        for key in ["input_ids", "attention_mask", "labels"]:
            if key in features[0]:
                batch[key] = torch.stack([
                    f[key] if isinstance(f[key], torch.Tensor) else torch.tensor(f[key], dtype=torch.long)
                    for f in features
                ])

        return batch

## 🔤 4. Токенизация и выравнивание меток

In [18]:
def find_label_for_span(word_idx: int, annotations,
                        tokenizer, text: str) -> str:
    """
    Ищет метку для слова с данным индексом.
    word_idx: порядковый номер слова в тексте (0,1,2,…).
    annotations: список троек (start_char, end_char, label).
    tokenizer, text: нужны, чтобы определить границы слова в символах.
    """
    # Сперва получим список слов и их границы в символах
    assert tokenizer is not None and text is not None, \
        "Нужны tokenizer и исходный text для определения границ."
    words = text.split()
    # Определяем границы слова word_idx
    char_pos = 0
    for idx, w in enumerate(words):
        start = char_pos
        end = start + len(w)
        if idx == word_idx:
            # Ищем, попадает ли этот span в любую аннотацию
            for a_start, a_end, a_label in annotations:
                if not (end <= a_start or start >= a_end):
                    return a_label
            return 'O'
        # двигаем позицию дальше (+1 за пробел)
        char_pos = end + 1

    return 'O'

In [19]:
def tokenize_and_align_labels_enhanced(example, tokenizer, label_to_id, max_length=128, apply_augmentation=False):
    text = example['sample']
    annotations = example['annotation']

    # 1) Аугментация опечаток (если включена)
    if apply_augmentation and random.random() < 0.3:
        words = text.split()
        augmented = []
        for word in words:
            if any(lbl.startswith('B-BRAND') for _, _, lbl in annotations) and random.random() < 0.2:
                augmented.append(augment_typo(word))
            else:
                augmented.append(word)
        text = ' '.join(augmented)

    # 2) Токенизация BERT
    encoding = tokenizer(
        text,
        truncation=True,
        padding='max_length',
        max_length=max_length,
        return_offsets_mapping=True,
        return_tensors='pt'
    )
    tokens = tokenizer.convert_ids_to_tokens(encoding['input_ids'][0])
    word_ids = encoding.word_ids(batch_index=0)

    # 3) Символьные ID
    char_ids_list = []
    for token in tokens:
        if token in tokenizer.all_special_tokens:
            char_ids_list.append([0]*20)
        else:
            clean = token[2:] if token.startswith('##') else token
            char_ids_list.append(char_tensor(clean))
    char_ids = torch.tensor(char_ids_list)  # [seq_len, max_char_len]

    # 4) Выравнивание меток в формате BIO на всех субтокенах
    labels = []
    prev_word_idx = None
    prev_label = 'O'

    for idx, word_idx in enumerate(word_ids):
        if word_idx is None:
            labels.append(-100)
            prev_word_idx = None
            prev_label = 'O'
            continue

        # определяем исходную метку слова
        span_label = find_label_for_span(
            word_idx, annotations,
            tokenizer=tokenizer, text=example['sample']
        )
        # если это первый субтокен слова
        if word_idx != prev_word_idx:
            label = span_label
        else:
            # наследуем тип, заменяя B- на I- или сохраняя I-
            if prev_label.startswith('B-'):
                label = 'I-' + prev_label.split('-', 1)[1]
            elif prev_label.startswith('I-'):
                label = prev_label
            else:
                label = 'O'
        labels.append(label_to_id.get(label, label_to_id['O']))

        prev_word_idx = word_idx
        prev_label = label

    # 5) Возврат батча
    return {
        'input_ids':      encoding['input_ids'][0],
        'attention_mask': encoding['attention_mask'][0],
        'char_ids':       char_ids,
        'labels':         torch.tensor(labels)
    }


In [20]:
def prepare_datasets_word_level(train_df, val_df, tokenizer, config, test_size=0.2, random_state=42):
    """
    Подготовка Dataset с word-level токенизацией, где tokenize_and_align_labels_word_level
    вызывается для каждого примера по отдельности.
    """
    print(f"📊 Train: {len(train_df)}, Validation: {len(val_df)}")

    # 2. Функция-обертка для одного примера
    def tokenize_example(example):
        # example["sample"] — строка текста
        # example["annotation"] — список аннотаций (start, end, label)
        single = {"sample": example["sample"], "annotation": example["annotation"]}
        tokenized = tokenize_and_align_labels_enhanced(
            single, tokenizer, config.LABEL_TO_ID, config.MAX_LENGTH
        )
        # возвращаем dict с полями input_ids, attention_mask, labels
        return tokenized

    # 3. Токенизация train и val по примерам
    print("🔄 Токенизация training данных...")
    train_records = [tokenize_example(row) for _, row in train_df.iterrows()]

    print("🔄 Токенизация validation данных...")
    val_records = [tokenize_example(row) for _, row in val_df.iterrows()]

    # 4. Создание HuggingFace Dataset
    train_dataset = Dataset.from_list(train_records)
    val_dataset = Dataset.from_list(val_records)

    return {"train": train_dataset, "validation": val_dataset}


## 🧠 5. BERT+BiLSTM+CRF модель

In [21]:
def create_transition_constraints(label_to_id):
    '''Создает матрицу ограничений для CRF переходов'''
    num_labels = len(label_to_id)
    constraints = torch.zeros((num_labels, num_labels), dtype=torch.bool)

    # Разрешаем все переходы по умолчанию
    constraints.fill_(True)

    # Запрещаем переходы O -> I-TYPE, O -> I-BRAND
    # I-TYPE может следовать только после B-TYPE или I-TYPE
    # I-BRAND может следовать только после B-BRAND или I-BRAND

    for from_label, from_id in label_to_id.items():
        for to_label, to_id in label_to_id.items():
            # Запрещаем O -> I-*
            if from_label == 'O' and to_label.startswith('I-'):
                constraints[from_id, to_id] = False

            # Запрещаем B-TYPE -> I-BRAND и наоборот
            if from_label.startswith('B-TYPE') and to_label.startswith('I-BRAND'):
                constraints[from_id, to_id] = False
            if from_label.startswith('B-BRAND') and to_label.startswith('I-TYPE'):
                constraints[from_id, to_id] = False

            # Запрещаем I-TYPE -> I-BRAND и наоборот
            if from_label.startswith('I-TYPE') and to_label.startswith('I-BRAND'):
                constraints[from_id, to_id] = False
            if from_label.startswith('I-BRAND') and to_label.startswith('I-TYPE'):
                constraints[from_id, to_id] = False

    return constraints

In [52]:
class BertBiLSTMCRFForTokenClassification(nn.Module):
    """BERT+BiLSTM+CRF модель без CharCNN для стабильности"""

    def __init__(self, bert, config):
        super().__init__()
        self.config = config
        self.num_labels = len(config.LABELS)
        self.bert = bert

        # Размерности
        bert_hidden_size = bert.config.hidden_size
        bilstm_hidden_size = config.BILSTM_HIDDEN_SIZE
        num_layers = config.BILSTM_NUM_LAYERS
        dropout = config.BILSTM_DROPOUT


        # BERT с градиентами
        self.bert.train()
        for param in self.bert.parameters():
            param.requires_grad = True

        # BiLSTM слой
        self.bilstm = nn.LSTM(
            input_size=bert_hidden_size,
            hidden_size=bilstm_hidden_size,
            num_layers=config.BILSTM_NUM_LAYERS,
            bidirectional=True,
            batch_first=True,
            dropout=config.BILSTM_DROPOUT if config.BILSTM_NUM_LAYERS > 1 else 0
        )

        # Dropout и классификатор
        self.dropout = nn.Dropout(config.BILSTM_DROPOUT)
        self.classifier = nn.Linear(bilstm_hidden_size * 2, self.num_labels)  # *2 для bidirectional

        # CRF слой
        self.crf = CRF(self.num_labels, batch_first=True)

        # Инициализация весов
        self._init_weights()

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def _init_weights(self):
        # Инициализация BiLSTM
        for name, param in self.bilstm.named_parameters():
            if 'weight' in name:
                nn.init.xavier_uniform_(param)
            elif 'bias' in name:
                nn.init.zeros_(param)

        # Инициализация классификатора
        nn.init.xavier_uniform_(self.classifier.weight)
        nn.init.zeros_(self.classifier.bias)

    def forward(self, input_ids, attention_mask=None, labels=None, **kwargs):
        # 1. BERT embeddings
        bert_outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            return_dict=True
        )
        sequence_output = bert_outputs.last_hidden_state  # shape [batch, max_length, hidden_size]

        # 2. BiLSTM с упаковкой (pack/unpack)
        if attention_mask is not None:
            # Вычисляем реальные длины последовательностей
            lengths = attention_mask.sum(dim=1).cpu()  # shape [batch]
            # Упаковываем
            packed_seq = nn.utils.rnn.pack_padded_sequence(
                sequence_output,
                lengths,
                batch_first=True,
                enforce_sorted=False
            )
            # Проходим через BiLSTM
            bilstm_packed, _ = self.bilstm(packed_seq)
            # Распаковываем обратно до полной длины
            bilstm_output, _ = nn.utils.rnn.pad_packed_sequence(
                bilstm_packed,
                batch_first=True,
                total_length=sequence_output.size(1)  # <-- здесь важно указать total_length
            )
        else:
            bilstm_output, _ = self.bilstm(sequence_output)

        # 3. Dropout и классификация
        bilstm_output = self.dropout(bilstm_output)
        logits = self.classifier(bilstm_output)  # shape [batch, max_length, num_labels]

        # 4. Создаем маску для валидных токенов
        mask = attention_mask.bool() if attention_mask is not None else torch.ones_like(input_ids, dtype=torch.bool)
        if labels is not None:
            # Подготавливаем метки для CRF
            crf_labels = labels.clone()
            crf_labels[labels == -100] = 0  # O метка

            # Обновляем маску - исключаем позиции с -100
            crf_mask = (labels != -100) & mask
            crf_mask[:, 0] = True
            # Вычисляем CRF loss с проверкой на пустые последовательности
            try:
                loss = -self.crf(logits, crf_labels, mask=crf_mask, reduction='mean')

                # Проверка на NaN/inf
                if torch.isnan(loss) or torch.isinf(loss):
                    print(f"Warning: Invalid loss detected: {loss}")
                    loss = torch.tensor(0.0, requires_grad=True, device=loss.device)

            except Exception as e:
                print(f"CRF loss error: {e}")
                loss = torch.tensor(0.0, requires_grad=True, device=logits.device)

            return {"loss": loss, "logits": logits}
        else:
            return {"logits": logits}

    def predict(self, input_ids, attention_mask=None):
        """CRF декодирование для предсказаний"""
        with torch.no_grad():
            outputs = self.forward(input_ids, attention_mask)
            logits = outputs["logits"]
            mask = attention_mask.bool() if attention_mask is not None else torch.ones_like(input_ids, dtype=torch.bool)
            return self.crf.decode(logits, mask=mask)

    @classmethod
    def from_pretrained(cls, model_dir, config, **kwargs):
        """
        Загружает модель из папки model_dir.
        Ожидает там файлы:
         - pytorch_model.bin  (state_dict)
         - config.json        (конфиг BERT)
        config — ваш объект конфигурации с атрибутами LABELS, BILSTM_HIDDEN_SIZE и т.д.
        """
        # 1. Загружаем BERT
        cfg = AutoConfig.from_pretrained(
            model_dir,
            add_pooling_layer=False,
            return_dict=True
        )
        # 2. Загружаем дообученный BertModel
        bert = BertModel.from_pretrained(
            model_dir,
            config=cfg,
            ignore_mismatched_sizes=True
        )
        config.id2label = {idx: label for idx, label in enumerate(config.LABELS)}
        config.label2id = {label: idx for idx, label in enumerate(config.LABELS)}
        # 2. Создаём экземпляр модели
        model = cls(bert, config)

        # 3. Загружаем state_dict
        import torch
        state_dict = torch.load(f"{model_dir}/pytorch_model.bin", map_location="cpu")
        model.load_state_dict(state_dict, strict=False)

        return model

## 🏋️ 6. Настройка Trainer и метрик

In [23]:
class BiLSTMCRFTrainer(Trainer):
    def __init__(self, config, **kwargs):
        self.config = config
        super().__init__(**kwargs)

    def create_optimizer(self):
        """Создаем оптимизатор с разными learning rates"""
        model = self.model

        # Разделяем параметры по группам
        bert_params = []
        bilstm_params = []
        crf_params = []
        classifier_params = []

        for name, param in model.named_parameters():
            if not param.requires_grad:
                continue

            if name.startswith('bert'):
                bert_params.append(param)
            elif name.startswith('bilstm'):
                bilstm_params.append(param)
            elif name.startswith('crf'):
                crf_params.append(param)
            else:
                classifier_params.append(param)

        # Создаем группы параметров с разными learning rates
        optimizer_grouped_parameters = [
            {
                'params': bert_params,
                'lr': self.config.LEARNING_RATE,
                'weight_decay': self.args.weight_decay,
            },
            {
                'params': bilstm_params,
                'lr': self.config.BILSTM_LEARNING_RATE,
                'weight_decay': self.args.weight_decay,
            },
            {
                'params': crf_params + classifier_params,
                'lr': self.config.CRF_LEARNING_RATE,
                'weight_decay': self.args.weight_decay,
            }
        ]

        self.optimizer = AdamW(optimizer_grouped_parameters, eps=1e-8)
        return self.optimizer

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")

        outputs = model(
            input_ids=inputs["input_ids"],
            attention_mask=inputs.get("attention_mask"),
            labels=labels
        )

        loss = outputs["loss"]

        return (loss, outputs) if return_outputs else loss

    def prediction_step(self, model, inputs, prediction_loss_only, ignore_keys=None):
        inputs = self._prepare_inputs(inputs)
        labels = inputs.pop("labels")
        with torch.no_grad():
            # Вызываем forward, чтобы получить loss и logits
            outputs = model(
                input_ids=inputs["input_ids"],
                attention_mask=inputs.get("attention_mask"),
                labels=labels
            )
            loss = outputs["loss"]  # теперь loss не None
            # Декодируем предсказания через CRF
            raw_preds = model.predict(
                input_ids=inputs["input_ids"],
                attention_mask=inputs.get("attention_mask")
            )
            # Приводим к тензору с той же формой, что и labels
            max_len = labels.shape[1]
            padded_preds = [
                pred[:max_len] + [0] * max(0, max_len - len(pred))
                for pred in raw_preds
            ]
            preds_tensor = torch.tensor(padded_preds, device=labels.device)

        # Важно вернуть loss вместе с preds и labels
        if prediction_loss_only:
            return (loss, None, None)
        return (loss, preds_tensor, labels)

In [24]:
'''def compute_metrics(eval_pred):
    """Вычисление метрик для NER с акцентом на macro-F1."""
    predictions, labels = eval_pred

    # Преобразуем predictions и labels в списки меток
    true_predictions = []
    true_labels = []

    for i in range(len(predictions)):
        pred_seq = []
        label_seq = []

        for j in range(len(predictions[i])):
            if labels[i][j] != -100:  # Игнорируем padding
                pred_seq.append(config.ID_TO_LABEL.get(int(predictions[i][j]), 'O'))
                label_seq.append(config.ID_TO_LABEL.get(int(labels[i][j]), 'O'))

        true_predictions.append(pred_seq)
        true_labels.append(label_seq)

    # Используем seqeval для token-level метрик
    try:
        seqeval_metric = evaluate.load("seqeval")
        seqeval_results = seqeval_metric.compute(
            predictions=true_predictions,
            references=true_labels
        )
    except:
        seqeval_results = {
            "overall_precision": 0.0,
            "overall_recall": 0.0,
            "overall_f1": 0.0,
            "overall_accuracy": 0.0
        }

    # Вычисляем macro-метрики вручную
    def extract_entities(labels_seq):
        """Извлекает сущности из BIO последовательности."""
        entities = []
        current_entity = None

        for i, label in enumerate(labels_seq):
            if label.startswith('B-'):
                if current_entity:
                    entities.append(current_entity)
                current_entity = (i, i+1, label[2:])
            elif label.startswith('I-') and current_entity and label[2:] == current_entity[2]:
                current_entity = (current_entity[0], i+1, current_entity[2])
            else:
                if current_entity:
                    entities.append(current_entity)
                current_entity = None

        if current_entity:
            entities.append(current_entity)

        return set(entities)

    # Подсчет для каждого типа сущности
    entity_types = ['TYPE', 'BRAND']
    type_stats = {etype: {'tp': 0, 'fp': 0, 'fn': 0} for etype in entity_types}

    for pred_seq, true_seq in zip(true_predictions, true_labels):
        pred_entities = extract_entities(pred_seq)
        true_entities = extract_entities(true_seq)

        # Группируем по типам
        for etype in entity_types:
            pred_type = {e for e in pred_entities if e[2] == etype}
            true_type = {e for e in true_entities if e[2] == etype}

            type_stats[etype]['tp'] += len(pred_type & true_type)
            type_stats[etype]['fp'] += len(pred_type - true_type)
            type_stats[etype]['fn'] += len(true_type - pred_type)

    # Вычисляем macro-метрики
    precisions = []
    recalls = []
    f1s = []

    for etype in entity_types:
        tp = type_stats[etype]['tp']
        fp = type_stats[etype]['fp']
        fn = type_stats[etype]['fn']

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0

        precisions.append(precision)
        recalls.append(recall)
        f1s.append(f1)

    macro_precision = np.mean(precisions) if precisions else 0.0
    macro_recall = np.mean(recalls) if recalls else 0.0
    macro_f1 = np.mean(f1s) if f1s else 0.0

    return {
        'precision': seqeval_results["overall_precision"],
        'recall': seqeval_results["overall_recall"],
        'f1': seqeval_results["overall_f1"],
        'accuracy': seqeval_results["overall_accuracy"],
        'macro_precision': macro_precision,
        'macro_recall': macro_recall,
        'macro_f1': macro_f1,
        'type_precision': precisions[0] if len(precisions) > 0 else 0.0,
        'type_recall': recalls[0] if len(recalls) > 0 else 0.0,
        'type_f1': f1s[0] if len(f1s) > 0 else 0.0,
        'brand_precision': precisions[1] if len(precisions) > 1 else 0.0,
        'brand_recall': recalls[1] if len(recalls) > 1 else 0.0,
        'brand_f1': f1s[1] if len(f1s) > 1 else 0.0,
    }

print("✅ Trainer и метрики настроены!")'''

'def compute_metrics(eval_pred):\n    """Вычисление метрик для NER с акцентом на macro-F1."""\n    predictions, labels = eval_pred\n\n    # Преобразуем predictions и labels в списки меток\n    true_predictions = []\n    true_labels = []\n\n    for i in range(len(predictions)):\n        pred_seq = []\n        label_seq = []\n\n        for j in range(len(predictions[i])):\n            if labels[i][j] != -100:  # Игнорируем padding\n                pred_seq.append(config.ID_TO_LABEL.get(int(predictions[i][j]), \'O\'))\n                label_seq.append(config.ID_TO_LABEL.get(int(labels[i][j]), \'O\'))\n\n        true_predictions.append(pred_seq)\n        true_labels.append(label_seq)\n\n    # Используем seqeval для token-level метрик\n    try:\n        seqeval_metric = evaluate.load("seqeval")\n        seqeval_results = seqeval_metric.compute(\n            predictions=true_predictions,\n            references=true_labels\n        )\n    except:\n        seqeval_results = {\n            "ov

In [25]:
def compute_metrics(eval_pred):
    """Вычисление token- и entity-level метрик только для TYPE и BRAND."""
    predictions, labels = eval_pred

    # Собираем списки меток без паддинга
    true_preds, true_labels = [], []
    for pred_seq, label_seq in zip(predictions, labels):
        preds, refs = [], []
        for p, l in zip(pred_seq, label_seq):
            if l == -100:
                continue
            preds.append(config.ID_TO_LABEL[int(p)])
            refs.append(config.ID_TO_LABEL[int(l)])
        true_preds.append(preds)
        true_labels.append(refs)

    # Token-level через seqeval
    seqeval = evaluate.load("seqeval")
    tok = seqeval.compute(predictions=true_preds, references=true_labels)

    # Функция извлечения сущностей из BIO
    def extract_entities(seq):
        ents, cur = set(), None
        for i, lab in enumerate(seq):
            if lab.startswith("B-"):
                if cur:
                    ents.add(cur)
                cur = (i, i+1, lab[2:])
            elif lab.startswith("I-") and cur and lab[2:]==cur[2]:
                cur = (cur[0], i+1, cur[2])
            else:
                if cur:
                    ents.add(cur)
                cur = None
        if cur:
            ents.add(cur)
        return ents

    # Считаем TP/FP/FN для TYPE и BRAND
    types = ["TYPE","BRAND"]
    stats = {t:{"tp":0,"fp":0,"fn":0} for t in types}

    for preds, refs in zip(true_preds, true_labels):
        pred_ents = extract_entities(preds)
        true_ents = extract_entities(refs)
        for t in types:
            pset = {e for e in pred_ents if e[2]==t}
            tset = {e for e in true_ents if e[2]==t}
            stats[t]["tp"] += len(pset & tset)
            stats[t]["fp"] += len(pset - tset)
            stats[t]["fn"] += len(tset - pset)

    # Вычисляем F1 по каждому типу и макро
    f1s = []
    for t in types:
        tp, fp, fn = stats[t]["tp"], stats[t]["fp"], stats[t]["fn"]
        prec = tp/(tp+fp) if tp+fp>0 else 0.0
        rec  = tp/(tp+fn) if tp+fn>0 else 0.0
        f1   = 2*prec*rec/(prec+rec) if prec+rec>0 else 0.0
        f1s.append(f1)

    macro_f1 = float(np.mean(f1s))

    return {
        "precision": tok["overall_precision"],
        "recall":    tok["overall_recall"],
        "f1":        tok["overall_f1"],
        "accuracy":  tok["overall_accuracy"],
        "macro_f1":  macro_f1,
        "type_f1":   f1s[0],
        "brand_f1":  f1s[1],
    }

## 🚀 7. Обучение модели

In [26]:
def create_optimizer(model, config):
    """
    Создает AdamW оптимизатор с разными LR для BERT и CRF
    и разделением на decay / no_decay.
    """
    no_decay = {"bias", "LayerNorm.weight"}

    # Группируем параметры BERT|
    bert_decay = []
    bert_no_decay = []
    for name, param in model.bert.named_parameters():
        if not param.requires_grad:
            continue
        if any(nd in name for nd in no_decay):
            bert_no_decay.append(param)
        else:
            bert_decay.append(param)

    # Параметры CRF (все через один LR и decay)
    crf_params = [param for name, param in model.named_parameters() if "crf" in name and param.requires_grad]

    # Составляем списки групп
    optimizer_grouped_parameters = [
        {
            "params": bert_decay,
            "lr": config.LEARNING_RATE,
            "weight_decay": config.WEIGHT_DECAY,
        },
        {
            "params": bert_no_decay,
            "lr": config.LEARNING_RATE,
            "weight_decay": 0.0,
        },
        {
            "params": crf_params,
            "lr": config.CRF_LEARNING_RATE,
            "weight_decay": config.WEIGHT_DECAY,
        },
    ]

    return AdamW(optimizer_grouped_parameters, eps=1e-8)

In [27]:
# Конвертируем и токенизируем
print("=" * 70)
print("🔤 ТОКЕНИЗАЦИЯ И ВЫРАВНИВАНИЕ")
print("=" * 70)

# Загружаем токенизатор
tokenizer = AutoTokenizer.from_pretrained(config.MODEL_NAME)
print(f"📚 Токенизатор: {config.MODEL_NAME}")

# Создаем Dataset
tokenized_datasets = DatasetDict(prepare_datasets_word_level(train, val, tokenizer,
                                                             config, test_size=0.1))

print("✅ Токенизация завершена успешно!")

# Проверяем распределение меток в финальном датасете
label_counts = Counter()
for item in tokenized_datasets['train']:
    for label_id in item['labels']:
        if label_id != -100:
            label_counts[config.ID_TO_LABEL[label_id]] += 1

print("\n📊 Распределение меток в обучающем наборе:")
total_labels = sum(label_counts.values())
for label, count in label_counts.most_common():
    print(f"  {label}: {count:,} ({count/total_labels*100:.1f}%)")

🔤 ТОКЕНИЗАЦИЯ И ВЫРАВНИВАНИЕ
📚 Токенизатор: ./tiny2_v5
📊 Train: 33334, Validation: 5578
🔄 Токенизация training данных...
🔄 Токенизация validation данных...
✅ Токенизация завершена успешно!

📊 Распределение меток в обучающем наборе:
  I-TYPE: 37,228 (34.6%)
  B-TYPE: 24,365 (22.7%)
  I-BRAND: 21,372 (19.9%)
  B-BRAND: 14,165 (13.2%)
  O: 10,376 (9.7%)


In [28]:
indices = list(range(10))
batch = tokenized_datasets['train'].select(indices)

for i, example in enumerate(batch):
    print(f"=== Пример {i} ===")
    input_ids = example["input_ids"]
    labels    = example["labels"]
    tokens    = tokenizer.convert_ids_to_tokens(input_ids)
    print("input_ids:", input_ids)
    print("labels:   ", labels)
    print("tokens:   ", tokens)
    print()

=== Пример 0 ===
input_ids: [2, 18646, 17991, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
labels:    [-100, 1, 2, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -10

In [30]:
# Инициализация модели
print("=" * 70)
print("🧠 ИНИЦИАЛИЗАЦИЯ BERT+CRF МОДЕЛИ")
print("=" * 70)
config.char_vocab_size = len(CHAR_VOCAB)
config.CRF_LEARNING_RATE = 5e-5  # Уменьшили с 1e-4
config.LEARNING_RATE = 2e-5      # Уменьшили с 3e-5
config.BATCH_SIZE = 16           # Уменьшили для стабильности
config.GRADIENT_ACCUMULATION_STEPS = 2
config.eos_token_id = tokenizer.eos_token_id
config.pad_token_id = tokenizer.pad_token_id
config.bos_token_id = getattr(tokenizer, 'bos_token_id', None)
config.unk_token_id = tokenizer.unk_token_id
# 1. Загружаем конфигурацию BERT без pooler
hf_config = AutoConfig.from_pretrained(
    config.MODEL_NAME,
    num_labels=len(config.LABELS),
    dropout=0.1,
    dd_pooling_layer=False,
    return_dict=True
)
hf_config.char_vocab_size = len(CHAR_VOCAB)
hf_config.label_to_id = config.LABEL_TO_ID


## 2. Загружаем BERT модель
bert_model = BertModel.from_pretrained(
    config.MODEL_NAME,
    config=hf_config,
    ignore_mismatched_sizes=True
)

# 3. Создаём BERT+BiLSTM+CRF модель
model = BertBiLSTMCRFForTokenClassification(bert_model, config)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

tokenizer = AutoTokenizer.from_pretrained(config.MODEL_NAME)
# Подсчет параметров
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"🔢 Всего параметров: {total_params:,}")
print(f"🎯 Обучаемых параметров: {trainable_params:,}")
print(f"📊 Количество меток: {len(config.LABELS)}")
print(f"🏷️ Метки: {config.LABELS}")
print(f"💻 Модель размещена на: {device}")

# Training Arguments
training_args = TrainingArguments(
    output_dir=config.OUTPUT_DIR,
    num_train_epochs=config.EPOCHS,
    per_device_train_batch_size=config.BATCH_SIZE,
    per_device_eval_batch_size=config.BATCH_SIZE,
    gradient_accumulation_steps=config.GRADIENT_ACCUMULATION_STEPS,
    learning_rate=config.CRF_LEARNING_RATE,
    weight_decay=config.WEIGHT_DECAY,
    warmup_ratio=config.WARMUP_RATIO,
    # Evaluation
    eval_strategy="steps",
    eval_steps=150,
    save_strategy="steps",
    save_steps=300,
    # Метрики и early stopping
    load_best_model_at_end=True,
    metric_for_best_model="macro_f1",
    greater_is_better=True,
    # Логирование
    logging_dir='./logs',
    logging_steps=50,
    # Оптимизации
    fp16=True,  # Смешанная точность для ускорения
    dataloader_pin_memory=True,
    dataloader_num_workers=2,
    # Сохранение
    save_total_limit=3,
    # Отключаем внешнее логирование
    report_to='none',
    # Для воспроизводимости
    seed=42,
    max_grad_norm=1.0,             # 🔧 Добавили gradient clipping
    save_safetensors=False,

)

# Создаем оптимизатор
optimizer = create_optimizer(model, config)

# Создаем scheduler
num_training_steps = len(tokenized_datasets["train"]) // (training_args.gradient_accumulation_steps * training_args.per_device_train_batch_size) * training_args.num_train_epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(training_args.warmup_ratio * num_training_steps),
    num_training_steps=num_training_steps
)

# Создаем data collator
data_collator = BiLSTMDataCollatorForTokenClassification(tokenizer)

# Инициализируем trainer с custom оптимизатором
trainer = BiLSTMCRFTrainer(
    config=config,  # Передаем config
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
print("=" * 70)
print("🚀 НАЧИНАЕМ ОБУЧЕНИЕ BERT+CRF МОДЕЛИ")
print("=" * 70)
print(f"📊 Train образцов: {len(tokenized_datasets['train']):,}")
print(f"🔍 Val образцов: {len(tokenized_datasets['validation']):,}")
print(f"⏰ Эпох: {config.EPOCHS}")
print(f"📈 BERT Learning rate: {config.LEARNING_RATE}")
print(f"🔗 CRF Learning rate: {config.CRF_LEARNING_RATE}")
print(f"📦 Batch size: {config.BATCH_SIZE}")
print(f"🎯 Цель: macro-F1 ≥ 0.8")
print(f"💻 Устройство: {device}")
print("=" * 70)

# Запускаем обучение
print("🚀 BERT+BiLSTM+CRF trainer готов к обучению!")
print(f"📊 BERT LR: {config.LEARNING_RATE}")
print(f"📊 BiLSTM LR: {config.BILSTM_LEARNING_RATE}")
print(f"📊 CRF LR: {config.CRF_LEARNING_RATE}")

try:
    # Обучение
    train_result = trainer.train()

    print("\n🎉 Обучение завершено!")
    print(f"📈 Финальный training loss: {train_result.training_loss:.4f}")

    # Сохранение модели
    tokenizer.save_pretrained(config.OUTPUT_DIR)
    trainer.save_model(config.OUTPUT_DIR)
    hf_config.save_pretrained(config.OUTPUT_DIR)
    # 2) Сохраняем веса
    torch.save(model.state_dict(), os.path.join(config.OUTPUT_DIR, "pytorch_model.bin"))

    print(f"💾 Модель сохранена в {config.OUTPUT_DIR}'")

    # Финальная оценка
    print("\n📊 Финальная оценка на валидационном наборе:")
    eval_result = trainer.evaluate()

    print(f"🎯 Macro-F1: {eval_result['eval_macro_f1']:.4f}")
    print(f"📈 Overall F1: {eval_result['eval_f1']:.4f}")
    print(f"🏷️  TYPE F1: {eval_result['eval_type_f1']:.4f}")
    print(f"🏪 BRAND F1: {eval_result['eval_brand_f1']:.4f}")

    if eval_result['eval_macro_f1'] >= 0.8:
        print("\n🎉🎉🎉 ПОЗДРАВЛЯЕМ! ЦЕЛЬ ДОСТИГНУТА! 🎉🎉🎉")
        print(f"✅ Macro-F1 = {eval_result['eval_macro_f1']:.4f} ≥ 0.8")
    else:
        print(f"\n📈 Прогресс: {eval_result['eval_macro_f1']/0.8*100:.1f}% от цели")
        print("💡 Рекомендации для улучшения:")
        print("   - Увеличить количество эпох")
        print("   - Настроить learning rate")
        print("   - Добавить больше данных для редких классов")

except KeyboardInterrupt:
    print("\n⚠️ Обучение прервано пользователем")
    print("📁 Частично обученная модель сохранена")

except Exception as e:
    print(f"\n❌ Ошибка во время обучения: {e}")
    print("🔍 Проверьте данные и конфигурацию")

print("\n" + "=" * 70)

🧠 ИНИЦИАЛИЗАЦИЯ BERT+CRF МОДЕЛИ


Some weights of BertModel were not initialized from the model checkpoint at ./tiny2_v5 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🔢 Всего параметров: 30,363,728
🎯 Обучаемых параметров: 30,363,728
📊 Количество меток: 5
🏷️ Метки: ['O', 'B-TYPE', 'I-TYPE', 'B-BRAND', 'I-BRAND']
💻 Модель размещена на: cuda
🚀 НАЧИНАЕМ ОБУЧЕНИЕ BERT+CRF МОДЕЛИ
📊 Train образцов: 33,334
🔍 Val образцов: 5,578
⏰ Эпох: 3
📈 BERT Learning rate: 2e-05
🔗 CRF Learning rate: 5e-05
📦 Batch size: 16
🎯 Цель: macro-F1 ≥ 0.8
💻 Устройство: cuda
🚀 BERT+BiLSTM+CRF trainer готов к обучению!
📊 BERT LR: 2e-05
📊 BiLSTM LR: 2e-05
📊 CRF LR: 5e-05


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Macro F1,Type F1,Brand F1
150,4.050300,3.327042,0.713791,0.731517,0.722545,0.830265,0.760923,0.753192,0.768653
300,1.243200,1.250968,0.872218,0.878530,0.875362,0.903377,0.878116,0.857587,0.898646
450,0.839300,0.920943,0.888842,0.896410,0.892610,0.919491,0.895180,0.878860,0.911499
600,0.816700,0.736349,0.913302,0.914857,0.914079,0.935661,0.915843,0.897625,0.934061
750,0.777100,0.655341,0.914975,0.920817,0.917887,0.941510,0.918896,0.897100,0.940692
900,0.634000,0.665660,0.917055,0.920959,0.919003,0.941052,0.919238,0.897927,0.940549
1050,0.679300,0.611386,0.922135,0.929332,0.925719,0.945008,0.926287,0.905976,0.946599
1200,0.589400,0.575737,0.929621,0.933447,0.931530,0.948621,0.930344,0.910328,0.950359
1350,0.559000,0.554165,0.933503,0.934298,0.933901,0.951775,0.932652,0.912692,0.952612
1500,0.542100,0.620032,0.922805,0.926210,0.924504,0.942371,0.925405,0.908123,0.942688



🎉 Обучение завершено!
📈 Финальный training loss: 0.8910
💾 Модель сохранена в ./bert-bilstm-crf-fixed'

📊 Финальная оценка на валидационном наборе:


🎯 Macro-F1: 0.9378
📈 Overall F1: 0.9378
🏷️  TYPE F1: 0.9204
🏪 BRAND F1: 0.9551

🎉🎉🎉 ПОЗДРАВЛЯЕМ! ЦЕЛЬ ДОСТИГНУТА! 🎉🎉🎉
✅ Macro-F1 = 0.9378 ≥ 0.8



## 🔮 8. Постпроцессинг с регулярными выражениями

In [112]:
# improved_postprocessing.py
import re


class ImprovedPostProcessingConfig:
    """Конфигурация для извлечения VOLUME и PERCENT"""
    VOLUME_UNITS = [
        r"л(?:итр(?:ов|а|ам|ы)?)?", r"мл", r"ml", r"l(?:iter)?",
        r"г(?:рамм(?:ов|а|ам|ы)?)?", r"кг", r"kg", r"g(?:ram)?",
        r"шт(?:ук(?:и|а)?)?", r"упак(?:овк(?:а|и)?)?", r"пачк(?:а|и)?",
        r"банк(?:а|и)?", r"бутыл(?:к(?:а|и)?)?", r"коробк(?:а|и)?",
        r"порци(?:я|й|и)?", r"доз(?:ы|а)?", r"капсул(?:а|ы)?",
        r"табл(?:етк(?:а|и)?)?", r"ампул(?:а|ы)?"
    ]
    PERCENT_WORDS = [
        r"процент(?:ов|а|ам|ы)?", r"жирност(?:ь|и|ью)?",
        r"содержани(?:е|я|ем)?", r"концентраци(?:я|и|ей)?",
        r"доля", r"часть"
    ]
    EXCLUDE_PATTERNS = [
        r"19\d{2}|20\d{2}", r"\d+\s*°[CF]?", r"\d{1,2}:\d{2}",
        r"\d+\.\d+\.\d+", r"\d+[-/]\d+[-/]\d+", r"\d{4,}",
        r"\d+\s*см", r"\d+\s*мм"
    ]


def create_volume_patterns():
    cfg = ImprovedPostProcessingConfig
    num = r"\d+(?:[.,]\d+)?"
    patterns = []
    for unit in cfg.VOLUME_UNITS:
        patterns.append(rf"({num})\s*{unit}\b")
        patterns.append(rf"{unit}\s+({num})")
    patterns += [
        r"\d+(?:[.,]\d+)?\s*x\s*\d+(?:[.,]\d+)?\s*(?:мл|л|г|кг)",
        r"\d+\s*по\s*\d+(?:[.,]\d+)?\s*(?:мл|л|г|кг)",
        r"\d+(?:[.,]\d+)?/\d+(?:[.,]\d+)?\s*(?:мл|л|г|кг)"
    ]
    return patterns


def create_percent_patterns():
    cfg = ImprovedPostProcessingConfig
    num = r"\d+(?:[.,]\d+)?"
    patterns = [
        rf"({num})\s*%", rf"%\s*({num})",
        rf"({num})\s*процент", rf"процент\s+({num})"
    ]
    for w in cfg.PERCENT_WORDS:
        patterns.append(rf"({num})\s*{w}")
        patterns.append(rf"{w}\s+({num})")
    return patterns


def is_excluded_number(text, match):
    grp = match.group()
    for pat in ImprovedPostProcessingConfig.EXCLUDE_PATTERNS:
        if re.fullmatch(pat, grp, re.IGNORECASE):
            return True
    start, end = match.span()
    ctx = text[max(0, start - 10):end + 10].lower()
    if any(x in ctx for x in ("год", "версия", "модель", "артикул", "код")):
        return True
    return False


def extract_volume_percent_spans(text):
    """Возвращает список (start,end,label) для regex."""
    ents = []
    for pat in create_percent_patterns():
        for m in re.finditer(pat, text, re.IGNORECASE):
            if not is_excluded_number(text, m):
                # разбиваем число и % на два span
                if "%" in m.group():
                    num_span = re.search(r"\d+[.,]?\d*", m.group()).span()
                    rel_start = m.start() + num_span[0]
                    rel_end = m.start() + num_span[1]
                    ents.append((rel_start, rel_end, "PERCENT"))
                    # % символ
                    pct_pos = text.find("%", m.start(), m.end())
                    if pct_pos != -1:
                        ents.append((pct_pos, pct_pos + 1, "PERCENT"))
                else:
                    ents.append((m.start(), m.end(), "PERCENT"))
    for pat in create_volume_patterns():
        for m in re.finditer(pat, text, re.IGNORECASE):
            if not is_excluded_number(text, m):
                ents.append((m.start(), m.end(), "VOLUME"))
    # удаляем перекрытия PERCENT>VOLUME, длинные>короткие
    ents = sorted(ents, key=lambda x: (0 if x[2]=="PERCENT" else 1, -(x[1]-x[0]), x[0]))
    final = []
    for s,e,l in ents:
        if not any(not (e<=rs or s>=re_) for rs,re_,_ in final):
            final.append((s,e,l))
    return sorted(final, key=lambda x:x[0])


def apply_postprocessing(text, bert_ents):
    """
    Объединяет BERT- и regex-сущности.
    Возвращает список кортежей (start, end, label) по словам text.split().
    """
    # 1. Получаем слова и их spans
    words = text.split()
    spans = []
    pos = 0
    for w in words:
        start = text.find(w, pos)
        end = start + len(w)
        spans.append((start, end))
        pos = end

    # 2. Получаем BIO-метки из BERT по span index
    bert_map = { (ent['start'], ent['end']): ent['entity_group'] for ent in bert_ents }
    labels = ["O"] * len(words)
    for i,(s,e) in enumerate(spans):
        if (s,e) in bert_map:
            labels[i] = "B-"+bert_map[(s,e)]
        # попытка найти вложенные I-
        # но BERT уже дает span по слову

    # 3. Добавляем regex-entities, не затрагивая уже аннотированные
    for rs,re,label in extract_volume_percent_spans(text):
        for i,(s,e) in enumerate(spans):
            if s<=rs<e:
                if labels[i]=="O":
                    labels[i] = "B-"+label
                break
        # для I- если span > одного слова не нужно

    # 4. Возвращаем разметку по словам
    result = []
    for (s,e),lab in zip(spans, labels):
        result.append((s, e, lab))
    return result


In [113]:
def predict_with_custom_model(text, model, tokenizer, config, device):
    """
    Предсказание для BertBiLSTMCRF: возвращает список словарей
    [{'start': int, 'end': int, 'entity': str}, ...] для каждого слова.
    """
    # 1. Токенизация с offset_mapping
    encoding = tokenizer(
        text,
        truncation=True,
        padding='max_length',
        max_length=config.MAX_LENGTH,
        return_tensors='pt',
        return_offsets_mapping=True
    )
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    offsets = encoding['offset_mapping'][0].tolist()

    # 2. Предсказание CRF
    model.eval()
    with torch.no_grad():
        predictions = model.predict(input_ids=input_ids, attention_mask=attention_mask)

    # 3. Нет предсказаний?
    if not predictions or not predictions[0]:
        return []

    # 4. Декодирование меток по токенам
    pred_ids = predictions[0]
    decoded = [config.ID_TO_LABEL[id_] for id_ in pred_ids]

    # 5. Сплиты исходных слов и их spans
    words = text.split()
    word_spans = []
    pos = 0
    for w in words:
        start = text.find(w, pos)
        end = start + len(w)
        word_spans.append((start, end))
        pos = end

    # 6. Токен → слово
    token_to_word = []
    for (s, e) in offsets:
        if s == e == 0:
            token_to_word.append(None)
            continue
        mid = (s + e) / 2
        wi = next((i for i, (ws, we) in enumerate(word_spans) if ws <= mid < we), None)
        token_to_word.append(wi)

    # 7. Для каждого слова — первая токен-метка != "O"
    word_labels = ["O"] * len(words)
    for token_label, wi in zip(decoded, token_to_word):
        if wi is None:
            continue
        if word_labels[wi] == "O" and token_label != "O":
            word_labels[wi] = token_label

    # 8. Результат: список словарей с ключами 'start','end','entity'
    result = []
    for (start, end), label in zip(word_spans, word_labels):
        result.append({
            'start': start,
            'end': end,
            'entity_group': label
        })
    return result

In [114]:
def debug_model_prediction(text, model, tokenizer, config, device):
    """Функция для отладки предсказаний модели"""

    print(f"\nОТЛАДКА ДЛЯ ТЕКСТА: '{text}'")
    print("-" * 30)

    # 1. Токенизация
    encoding = tokenizer(
        text,
        truncation=True,
        padding='max_length',
        max_length=config.MAX_LENGTH,
        return_tensors='pt'
    )

    print(f"Input IDs: {encoding['input_ids'][0][:10]}...")
    print(f"Attention mask: {encoding['attention_mask'][0][:10]}...")

    # 2. Токены для отладки
    tokens = tokenizer.convert_ids_to_tokens(encoding['input_ids'][0])
    print(f"Tokens: {tokens[:10]}...")

    # 3. Перемещение на device
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    print(f"Device: {input_ids.device}")
    print(f"Input shape: {input_ids.shape}")

    # 4. Предсказание
    model.eval()
    try:
        with torch.no_grad():
            # Сначала пробуем forward
            outputs = model.forward(input_ids=input_ids, attention_mask=attention_mask)
            print(f"Forward успешен, logits shape: {outputs['logits'].shape}")

            # Теперь пробуем predict
            predictions = model.predict(input_ids=input_ids, attention_mask=attention_mask)
            print(f"Predict успешен, predictions: {type(predictions)}")

            if predictions:
                print(f"Predictions length: {len(predictions)}")
                if len(predictions) > 0:
                    print(f"First prediction length: {len(predictions[0])}")
                    print(f"First few labels: {predictions[0][:10]}")

                    # Декодируем метки
                    pred_labels = predictions[0]
                    decoded_labels = []
                    for label_id in pred_labels[:10]:
                        label = config.ID_TO_LABEL.get(label_id, f'UNK_{label_id}')
                        decoded_labels.append(label)
                    print(f"Decoded labels: {decoded_labels}")

                    return predictions
            else:
                print("Predictions is empty!")
                return None

    except Exception as e:
        print(f"ОШИБКА в predict: {e}")
        import traceback
        traceback.print_exc()
        return None

def test_with_debug():
    """Тестирование с отладкой"""

    print("ОТЛАДОЧНОЕ ТЕСТИРОВАНИЕ МОДЕЛИ")
    print("=" * 50)

    # Загрузка модели
    model_name = config.OUTPUT_DIR

    print(f"Загружаем модель из: {model_name}")

    try:
        test_tokenizer = AutoTokenizer.from_pretrained(model_name)
        print("Tokenizer загружен успешно")

        # ВАЖНО: правильная загрузка модели с config
        test_model = BertBiLSTMCRFForTokenClassification.from_pretrained(model_name, config)
        print("Модель загружена успешно")

        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        test_model.to(device)
        test_model.eval()

        print(f"Модель переведена на: {device}")

        # Проверяем config
        print(f"Config.MAX_LENGTH: {config.MAX_LENGTH}")
        print(f"Config.ID_TO_LABEL exists: {hasattr(config, 'ID_TO_LABEL')}")
        if hasattr(config, 'ID_TO_LABEL'):
            print(f"ID_TO_LABEL: {config.ID_TO_LABEL}")

        # Тестовый текст
        text = "молоко без сахара"

        predictions = debug_model_prediction(text, test_model, test_tokenizer, config, device)

        if predictions:
            print("\nПредсказания получены успешно!")
        else:
            print("\nПредсказания НЕ получены!")

    except Exception as e:
        print(f"КРИТИЧЕСКАЯ ОШИБКА: {e}")
        import traceback
        traceback.print_exc()

# Запустите эту функцию
test_with_debug()


ОТЛАДОЧНОЕ ТЕСТИРОВАНИЕ МОДЕЛИ
Загружаем модель из: ./bert-bilstm-crf-fixed
Tokenizer загружен успешно
Модель загружена успешно
Модель переведена на: cuda
Config.MAX_LENGTH: 128
Config.ID_TO_LABEL exists: True
ID_TO_LABEL: {0: 'O', 1: 'B-TYPE', 2: 'I-TYPE', 3: 'B-BRAND', 4: 'I-BRAND'}

ОТЛАДКА ДЛЯ ТЕКСТА: 'молоко без сахара'
------------------------------
Input IDs: tensor([    2, 38747,  2399, 38432,     3,     0,     0,     0,     0,     0])...
Attention mask: tensor([1, 1, 1, 1, 1, 0, 0, 0, 0, 0])...
Tokens: ['[CLS]', 'молоко', 'без', 'сахара', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']...
Device: cuda:0
Input shape: torch.Size([1, 128])
Forward успешен, logits shape: torch.Size([1, 128, 5])
Predict успешен, predictions: <class 'list'>
Predictions length: 1
First prediction length: 5
First few labels: [0, 1, 0, 0, 0]
Decoded labels: ['O', 'B-TYPE', 'O', 'O', 'O']

Предсказания получены успешно!


In [115]:
print("✅ Функции постпроцессинга готовы!")

✅ Функции постпроцессинга готовы!


## 🧪 9. Тестирование и демонстрация

In [118]:
def test_model_all_word_labels(model, tokenizer, config, device='cpu'):
    """
    Тестирование модели с выводом меток для всех слов.
    """
    test_texts = [
        'молоко без сахара', 'j7 сок',
              'сметана домик 7 %', 'конфеты m&ms',
              'колбас красна цена', 'торог',
              'кола 2л', 'кола 2 литра',
              'сметана домик 7% 2 литра', 'пиво хейнекен',
              'печенье милка', 'шоколад россия',
              'шоколад alpen', 'сметана домик 7',
              'сок 5 литра', 'напиток fresh тропический',
              'красная цена кукуруза', 'белоручка реди',
              'масло сливочное экомилк', 'сливки 33 %', 'сливки для взбиванмя',
              'сок сады придонья 2 литра', 'сосиски из курицы',
              'соус длч спагетти', 'сулугуни киржачский',
              'сок 5 × 0,33 л', 'сок 5 по 0,33 л',
              'кеф 3%', 'кифири danone 1,5 %', 'три процента',
              'жопа в деревне', 'йогурт в деревне', 'йогурт домик',
              'шоколад альпин голд', 'сыр с', 'молоко пармалат 5 % 500мл',
            'фрешбар'
    ]

    print("🧪 Тестирование с метками для каждого слова:")
    print("=" * 80)

    for i, text in enumerate(test_texts, 1):
        print(f"\n{i}. Текст: '{text}'")

        try:
            # Получаем метки для всех слов (включая постпроцессинг)
            all_word_labels = extract_volume_percent_entities(text)
            bert_entities = predict_with_custom_model(text, model, tokenizer, config, device)
            final = apply_postprocessing(text, bert_entities)

            # Форматируем вывод как в вашем примере
            result_string = f"{text}; {all_word_labels}"
            print(f"   Результат: {result_string}")

            # Дополнительно показываем каждое слово с меткой
            print("   Детализация:")
            words = text.split()
            for j, ((start, end, label), word) in enumerate(zip(all_word_labels, words)):
                emoji = {'TYPE': '📦', 'BRAND': '🏷️', 'VOLUME': '⚖️', 'PERCENT': '📊', 'O': '⚪'}.get(label, '🔖')
                print(f"     {emoji} '{word}' → ({start}, {end}, '{label}')")

        except Exception as e:
            print(f"   ❌ Ошибка: {e}")

    print("\n" + "=" * 80)


In [119]:
# Загрузка обученной модели
print("=" * 70)
print("🔮 ТЕСТИРОВАНИЕ ОБУЧЕННОЙ МОДЕЛИ")
print("=" * 70)

model_name = config.OUTPUT_DIR
# Загружаем токенизатор из той же папки
test_tokenizer = AutoTokenizer.from_pretrained(model_name)

# Попытка загрузить лучшую модель
test_model = BertBiLSTMCRFForTokenClassification.from_pretrained(model_name, config)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
test_model.to(device)
'''test_model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
test_model.to(device)'''

# Тестирование
test_model_all_word_labels(test_model, test_tokenizer, config, device)

🔮 ТЕСТИРОВАНИЕ ОБУЧЕННОЙ МОДЕЛИ
🧪 Тестирование с метками для каждого слова:

1. Текст: 'молоко без сахара'
   Результат: молоко без сахара; []
   Детализация:

2. Текст: 'j7 сок'
   Результат: j7 сок; []
   Детализация:

3. Текст: 'сметана домик 7 %'
   Результат: сметана домик 7 %; [(14, 17, 'PERCENT')]
   Детализация:
     📊 'сметана' → (14, 17, 'PERCENT')

4. Текст: 'конфеты m&ms'
   Результат: конфеты m&ms; []
   Детализация:

5. Текст: 'колбас красна цена'
   Результат: колбас красна цена; []
   Детализация:

6. Текст: 'торог'
   Результат: торог; []
   Детализация:

7. Текст: 'кола 2л'
   Результат: кола 2л; [(5, 7, 'VOLUME')]
   Детализация:
     ⚖️ 'кола' → (5, 7, 'VOLUME')

8. Текст: 'кола 2 литра'
   Результат: кола 2 литра; [(5, 12, 'VOLUME')]
   Детализация:
     ⚖️ 'кола' → (5, 12, 'VOLUME')

9. Текст: 'сметана домик 7% 2 литра'
   Результат: сметана домик 7% 2 литра; [(15, 18, 'PERCENT')]
   Детализация:
     📊 'сметана' → (15, 18, 'PERCENT')

10. Текст: 'пиво хейнекен'
 

# Генерация csv для submission

In [68]:
model_name = config.OUTPUT_DIR

df = pd.read_csv('./submission_true.csv', sep=';')

test_tokenizer = AutoTokenizer.from_pretrained(model_name)

# Попытка загрузить лучшую модель
test_model = BertCRFForTokenClassification.from_pretrained(model_name)
test_model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
test_model.to(device)

preds = []
for text in df['sample']:
    ents = predict_with_postprocessing(text, test_model, test_tokenizer, config, device)
    preds.append(ents)

df['annotation'] = preds
df.to_csv('submission.csv', sep=';', index=False)

print("Сохранено: submission.csv")

NameError: name 'BertCRFForTokenClassification' is not defined